In [1]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install tensorflow
!pip install torch
!pip install transformers
!pip install spacy
!pip install matplotlib

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.checkpoint import checkpoint
import re
import numpy as np
import pandas as pd
import torch
import spacy
from bisect import bisect_left, bisect_right
from torch import nn
from transformers import BertTokenizer, BertModel
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import ast

/Users/kissmihaly/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
df_annotations = pd.read_csv('annotations')

df_comments= pd.read_csv('comments')

df_spans= pd.read_csv('spans.csv')


print(df_annotations.head(5))
print(df_comments.head(5))
print(df_spans.head(5))

   annotation  comment_id  worker country  all toxic  not toxic
0           0     5167187     868     USA      False      False
1           1     5167187    1316     USA      False      False
2           2     5167187    1295     USA      False       True
3           3     5167187    2856     USA      False      False
4           4     5521110     418     VEN       True       True
   comment_id                                       comment_text
0      239607  Yet call out all Muslims for the acts of a few...
1      239612  This bitch is nuts. Who would read a book by a...
2      240311                                   You're an idiot.
3      240400  Nincompoop, that's a nice one! I'm partial to ...
4      240461  testing purposes:\n\nyou are an idiot and i ca...
   annotation    type  start  end
0           0  Insult    133  139
1           1  Insult     84   92
2           1  Insult    118  124
3           1  Insult    126  131
4           1  Insult    133  147


In [14]:
merge = pd.merge(df_annotations, df_spans, on='annotation', how='inner')
df_all = pd.merge(merge, df_comments, on='comment_id', how='inner')
df_all['word']=""
df_all=df_all.head(800)

for i in range(0, 799):
  str1=df_all['comment_text'][i]
  str2=""
  for j in range(df_all['start'][i],df_all['end'][i]):
    if df_all['end'][i]<=len(str1):
      str2=str2+str1[j]
  df_all.at[i, 'word'] = str2



In [15]:
df_all = df_all[df_all['start'] <= df_all['end']]
df_all = df_all[df_all['word'].notna() & (df_all['word'] != '')]
df_all['word'] = df_all['word'].apply(lambda x: x if len(x.split()) == 1 else None)
df_all.dropna(subset=['word'], inplace=True)
df_all = df_all.reset_index(drop=True)
df_all

,annotation,comment_id,worker,country,all toxic,not toxic,type,start,end,comment_text,word
0,0,5167187,868,USA,False,False,Insult,133,139,That's right. They are not normal. And I am st...,sexist
1,1,5167187,1316,USA,False,False,Insult,84,92,That's right. They are not normal. And I am st...,ABNORMAL
2,1,5167187,1316,USA,False,False,Insult,118,124,That's right. They are not normal. And I am st...,racist
3,1,5167187,1316,USA,False,False,Insult,126,131,That's right. They are not normal. And I am st...,bigot
4,3,5167187,2856,USA,False,False,Insult,84,92,That's right. They are not normal. And I am st...,ABNORMAL
...,...,...,...,...,...,...,...,...,...,...,...
588,455,763785,1043,USA,False,False,Insult,32,37,grin and you are nothing what a moron you are,moron
589,456,763785,519,USA,False,False,Insult,32,37,grin and you are nothing what a moron you are,moron
590,457,498684,2585,USA,False,False,Insult,88,95,Exactly. I wonder if paid subscribers to the ...,suckers
591,458,498684,67,VEN,False,False,Profane/Obscene,88,95,Exactly. I wonder if paid subscribers to the ...,suckers


In [17]:
url = 'https://raw.githubusercontent.com/ipavlopoulos/toxic_spans/master/SemEval2021/data/tsd_test.csv'
df_test= pd.read_csv(url)
df_test = df_test.rename(columns={df_test.columns[0]: 'index'})
df_test["toxic"]=0
df_test=df_test.head(100)

In [18]:

tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

batch_size = 500

for start in range(0, df_test.shape[0], batch_size):
    end = min(start + batch_size, df_test.shape[0])
    batch = df_test["text"][start:end]

    tokenized_sentences = tokenizer.batch_encode_plus(
        batch.tolist(),
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    input_ids = tokenized_sentences["input_ids"]
    attention_mask = tokenized_sentences["attention_mask"]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    def checkpoint_forward(model, *inputs):
        return model(*inputs)

    with torch.no_grad():
        model = model.to(device)
        outputs = checkpoint(checkpoint_forward, model, input_ids, attention_mask)

    for i in range(start, end):
        scores = outputs.logits[i - start].cpu().tolist()
        is_toxic = float(np.max(scores) > 0.2)
        df_test.at[i, "toxic"] = is_toxic


/Users/kissmihaly/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [14]:
df_test

,index,text,toxic,pindex
0,"[84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...",That's right. They are not normal. And I am st...,1,"[118, 119, 120, 121, 122, 123, 126, 127, 128, ..."
1,"[81, 82, 83, 84, 85, 86]","""Watch people die from taking away their healt...",1,"[81, 82, 83, 84, 85, 86]"
2,[],tens years ago i contacted the PDR and suggest...,0,[]
3,[],The parallels between the ANC and the Sicilian...,0,[]
4,[],Intel Community: ‘How can we work for a Presid...,0,[]
...,...,...,...,...
1995,"[4, 5, 6, 7, 8, 70, 71, 72, 73, 74, 75, 76, 77...",hey loser change your name to something more a...,1,"[4, 5, 6, 7, 8, 70, 71, 72, 73, 74, 75, 76, 77..."
1996,"[23, 24, 25, 26, 27]",And you are a complete moron who obviously doe...,1,"[23, 24, 25, 26, 27, 80, 81, 82, 83, 84, 85, 8..."
1997,"[157, 158, 159, 160, 161, 162, 163, 164, 165, ...",Such vitriol from the left. Who would have th...,1,"[99, 100, 101, 102, 108, 109, 110, 111, 112, 1..."
1998,[],It is now time for most of you to expand your ...,0,[]


In [19]:
#TOXICWORDS.TXT
def predict_txt(path, df):
    df_test['pindex']=''
    with open(path, "r") as file:
        toxic_words = [word.strip() for word in file]

    for i in range(0, df.shape[0]):
        if df['toxic'][i]==1:
            sentence = df['text'][i]
            indexes = []
            for word in toxic_words:
                pattern = r"\b" + re.escape(word) + r"\b"
                matches = re.finditer(pattern, sentence, flags=re.IGNORECASE)
                for match in matches:
                    start_index = match.start()
                    end_index = match.end()
                    indexes.extend(list(range(start_index, end_index)))

            indexes.sort()
            df.at[i, 'pindex'] = indexes
        else:
            df.at[i, 'pindex'] = []
    return df

df_test=predict_txt("toxicwords.txt", df_test)

def f1(predictions, gold):
    if len(gold) == 0:
        return 1.0 if len(predictions) == 0 else 0.0
    if len(predictions) == 0:
        return 0.0
    predictions_set = set(predictions)
    gold_list = ast.literal_eval(gold)
    gold_set = set(gold_list)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom) / float(denom)



predicted = df_test["pindex"].tolist()
actual = df_test["index"].tolist()



f1_scores = []
for pred, act in zip(predicted, actual):
    f1_scores.append(f1(pred, act))

macro_avg_f1 = np.mean(f1_scores)

print("Macro-averaged F1-score:", macro_avg_f1)

Macro-averaged F1-score: 0.49886170280268227


-------------------------------------2. TXT----------------------------------

In [20]:
#TOXICWORDSLONG.TXT

df_test=predict_txt("toxicwordslong.txt", df_test)


predicted = df_test["pindex"].tolist()
actual = df_test["index"].tolist()

f1_scores = []
for pred, act in zip(predicted, actual):
    f1_scores.append(f1(pred, act))

macro_avg_f1 = np.mean(f1_scores)

print("Macro-averaged F1-score:", macro_avg_f1)




Macro-averaged F1-score: 0.38196479337490424


----------------------------------- 3. teszt -----------------------------------

In [12]:
def delete_duplicate_lines(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    unique_lines = list(set(lines))
    num_deleted_lines = len(lines) - len(unique_lines)

    with open(filename, 'w') as file:
        for line in unique_lines:
            file.write(line)

    return num_deleted_lines

filename = 'tw2.txt'

deleted_lines = delete_duplicate_lines(filename)
print(f"Az eltávolított sorok száma: {deleted_lines}")


Az eltávolított sorok száma: 0


In [21]:
#TW2.TXT

df_test=predict_txt("tw2.txt", df_test)


predicted = df_test["pindex"].tolist()
actual = df_test["index"].tolist()

f1_scores = []
for pred, act in zip(predicted, actual):
    f1_scores.append(f1(pred, act))

macro_avg_f1 = np.mean(f1_scores)

print("Macro-averaged F1-score:", macro_avg_f1)



Macro-averaged F1-score: 0.49474405574385877


-----------------------------------BERT MODELL--------------------------

In [26]:
import random

df_2=pd.read_csv('data.csv')

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [27]:
def toxic_tokens(df):
    for index, sample in df.iterrows():
        spans = eval(sample['spans'])
        correct_spans = spans.copy()
        chars = list(sample['text'])
        for i, char in enumerate(chars):
            if i == 0:
                continue
            if (i in spans) and (i - 1 not in spans) and (chars[i - 1].isalnum()) and (char.isalnum()):
                correct_spans.append(i - 1)
            elif (i - 1 in spans) and (i not in spans) and (chars[i - 1].isalnum()) and (char.isalnum()):
                correct_spans.append(i)
        correct_spans.sort()
        sample['spans'] = correct_spans

    df['toxic_tokens'] = [list() for x in range(len(df.index))]
    for _, sample in df_2.iterrows():
            toxic = ''
            for i, char in enumerate(list(sample["text"])):
                if i in sample["spans"]:
                    toxic += char
                elif len(toxic):
                    sample['toxic_tokens'].append(toxic)
                    toxic = ''
            if toxic:
                sample['toxic_tokens'].append(toxic)
    return df

df_2=toxic_tokens(df_2)

In [23]:
nlp = spacy.blank("en")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [24]:
def ranges(nums):
    nums = sorted(set(nums))
    gaps = [[s, e] for s, e in zip(nums, nums[1:]) if s + 1 < e]
    edges = iter(nums[:1] + sum(gaps, []) + nums[-1:])
    return list(zip(edges, edges))

def create_token_labels(df):
    text = nlp(df['text'])
    token_start = [token.idx for token in text]
    token_end = [token.idx + len(token) - 1 for token in text]
    toxic_ranges = ranges(df['spans'])
    l = len(df['text'])
    for range in toxic_ranges:
        start, end = range
        if end >= l:
            end = l - 1
        while start < l and df['text'][start] == ' ':
            start += 1
        while end >= 0 and df['text'][end] == ' ':
            end -= 1
        start = token_start[bisect_right(token_start, start) - 1]
        end = token_end[bisect_left(token_end, end)]
        if start >= end:
            print('Error:', df['text'])
            continue
        token_span = text.char_span(start, end + 1)
        for token in token_span:
                token.ent_type_ = 'toxic'

    bert_tokens = []
    token_labels = []
    for token in text:
        bert_subtokens = tokenizer.tokenize(token.text)
        bert_tokens += bert_subtokens
        token_labels += [int(token.ent_type_ == 'toxic') for _ in bert_subtokens]

    return bert_tokens, token_labels


def bert_token(df):
    bert_tokens_list = []
    token_labels_list = []

    for _, sample in df.iterrows():
        bert_tokens, token_labels = create_token_labels(sample)
        bert_tokens_list.append(bert_tokens)
        token_labels_list.append(token_labels)

    df['bert_tokens'] = bert_tokens_list
    df['token_labels'] = token_labels_list

    return df

df_2 = bert_token(df_2)

In [28]:
df_test2=df_test
df_test2['bert_tokens'] = [list() for x in range(len(df_test2.index))]

for _, sample in df_test2.iterrows():
    text = nlp(sample['text'])
    for token in text:
        sample['bert_tokens'] += tokenizer.tokenize(token.text)

In [17]:
maxlen_train = max(len(x) for x in df_2['bert_tokens'])
maxlen_test = max(len(x) for x in df_test2['bert_tokens'])
maxlen = max(maxlen_train, maxlen_test)

train_tokens = [['[CLS]'] + t[:maxlen - 2] + ['[SEP]'] for t in df_2['bert_tokens']]
test_tokens = [['[CLS]'] + t[:maxlen - 2] + ['[SEP]'] for t in df_test2['bert_tokens']]

len(train_tokens), len(test_tokens)


(7939, 2000)

In [ ]:
def padding(tokens, max_len=maxlen):
    tokens_len = len(tokens)
    pad_len = max(0, max_len - tokens_len)
    return (
        pad_sequences([tokens], maxlen=max_len, truncating="post", padding="post", dtype="int"),
        np.concatenate([np.ones(tokens_len, dtype="int"), np.zeros(pad_len, dtype="int")], axis=0)
    )

def token_and_mask(tokens):
    token_ids, masks = [], []

    for x in tokens:
        token_id, mask = padding(tokenizer.convert_tokens_to_ids(x))
        token_ids.append(token_id[0])
        masks.append(mask)

    token_ids = np.array(token_ids)
    masks = np.array(masks)

    return token_ids, masks

train_token_ids, train_masks = token_and_mask(train_tokens)
test_token_ids, test_masks = token_and_mask(test_tokens)


train_token_labels = list(map(lambda t: [0] + t[:maxlen - 2] + [0], df_2['token_labels']))

train_y = pad_sequences(train_token_labels, maxlen=maxlen, truncating="post", padding="post")[:, :, None]
train_y.shape, np.mean(train_y)


class BertClassifier(nn.Module):

    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.hidden = nn.Linear(bert.config.hidden_size, 64)
        self.hidden_activation = nn.LeakyReLU(0.1)
        self.output = nn.Linear(64, 1)
        self.output_activation = nn.Sigmoid()

    def forward(self, input_ids, attention_mask, labels=None):

        outputs = self.bert(input_ids, attention_mask=attention_mask)
        cls_output = outputs[0]
        cls_output = self.hidden(cls_output)
        cls_output = self.hidden_activation(cls_output)
        cls_output = self.output(cls_output)
        cls_output = self.output_activation(cls_output)
        criterion = nn.BCELoss()
        loss = 0
        if labels is not None:
            loss = criterion(cls_output, labels.float())
        return loss, cls_output

device = torch.device("cpu")
model = BertClassifier(BertModel.from_pretrained('bert-base-uncased')).to(device)

BATCH_SIZE = 16

train_dataset = TensorDataset(
    torch.tensor(train_token_ids), torch.tensor(train_masks), torch.tensor(train_y))
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(torch.tensor(test_token_ids), torch.tensor(test_masks))
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

optimizer = Adam(model.parameters(), lr=3e-6)
torch.cuda.empty_cache()


EPOCHS = 2
loss = nn.BCELoss()
total_len = len(train_token_ids)
batch_losses = []
model.train()

for epoch_num in range(EPOCHS):
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(tqdm(train_dataloader)):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        loss, _ = model(input_ids=token_ids, attention_mask=masks, labels=labels)

        train_loss += loss.item()

        model.zero_grad()
        loss.backward()

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()

        batch_losses.append(train_loss / (step_num + 1))


model.eval()

test_tokens = []
test_attention_masks = []
test_preds = []

with torch.no_grad():
    for step_num, batch_data in enumerate(tqdm(test_dataloader)):
        token_ids, masks = tuple(t.to(device) for t in batch_data)
        _, output = model(input_ids=token_ids, attention_mask=masks)
        test_tokens += token_ids.tolist()
        test_attention_masks += masks.tolist()
        test_preds += output[:, :, 0].tolist()





Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 125/125 [06:24<00:00,  3.07s/it]


In [18]:
def f1_scoremetric(predictions, gold, n):
    predictions = set(predictions)
    gold = set(gold)

    if len(gold) == 0:
        return {
            'f1': 1 if len(predictions) == 0 else 0
        }

    nom = 2 * len(predictions.intersection(gold))
    denom = len(predictions) + len(gold)

    if len(gold) == n:
        return {
            'f1': nom / denom
        }

    return {
        'f1': nom / denom
    }


def clean_f1(threshold):
    n = len(test_tokens)
    y_pred = []
    all_spans = []
    f1=0

    for i in range(n):
        reconstructed_text = ''
        spans = []
        original_text = df_test2.iloc[i]['text']
        original_text = original_text.replace('É', 'e')
        original_text = original_text.replace('\u200b', '')
        original_text = original_text.replace('ü', 'u')
        original_l = len(original_text)
        idx = 0
        tokens = test_tokens[i]
        n_tokens = len(tokens)

        j = 1
        last_token = None
        prev_token_toxic = False
        prev_token_prob = 0
        prev_idx = -1

        while j < n_tokens:
            while idx < original_l and original_text[idx].isspace():
                reconstructed_text += original_text[idx]
                idx += 1

            word = tokens[j]

            if word == '[SEP]':
                last_token = j - 1
                break

            if ord(original_text[idx]) == 65039:
                print('Problematic char at', idx, i)
                reconstructed_text += original_text[idx]
                idx += 1

            if word == '[UNK]':
                reconstructed_text += original_text[idx]
                idx += 1
                j += 1
                continue

            max_toxic_prob = test_preds[i][j]
            print(tokens)
            while j < n_tokens - 1 and tokens[j + 1].startswith('##'):
                j += 1
                word += tokens[j][2:]
                max_toxic_prob = max(max_toxic_prob, test_preds[i][j])

            word_l = len(word)
            y_pred += [min(prev_token_prob, max_toxic_prob) for _ in range(prev_idx + 1, idx)]

            if min(prev_token_prob, max_toxic_prob) >= threshold:
                spans += list(range(prev_idx + 1, idx))

            y_pred += [max_toxic_prob for _ in range(word_l)]
            is_toxic = (max_toxic_prob >= threshold)
            prev_token_prob = max_toxic_prob

            if word == original_text[idx: idx + word_l].lower():
                reconstructed_text += word

                if is_toxic:
                    spans += list(range(idx, idx + word_l))

                idx += word_l
                prev_idx = idx - 1
                prev_token_toxic = is_toxic
            else:
                print(word, original_text[idx: idx + word_l])

            j += 1

        while idx < original_l and original_text[idx].isspace():
            reconstructed_text += original_text[idx]
            idx += 1

        y_pred += [prev_token_prob for _ in range(prev_idx + 1, idx)]

        if reconstructed_text != original_text.lower():
            print('ISSUE', i)
        else:
            metrics = f1_scoremetric(spans, eval(df_test2.iloc[i]['index']), len(original_text))
            f1 += metrics['f1']


        all_spans.append(spans)

    return f1 / n, all_spans


In [ ]:
thresholds = [0.01 * x for x in range(100)]
f1_scores=[]
for x in tqdm(thresholds):
    scores = clean_f1(x)
    f1_scores.append(scores[0])

In [ ]:
df_test2["predicted"]=""
df_test2["predicted"] = [text_scores for uid, text_scores in zip(range(len(test_tokens)), clean_f1(0.5)[1])]
df_test2

In [242]:
predicted = df_test2["predicted"].tolist()
actual = df_test2["index"].tolist()

f1_scores2 = []
for pred, act in zip(predicted, actual):
    f1_scores2.append(f1(pred, act))

macro_avg_f1 = np.mean(f1_scores2)

print("Macro-averaged F1-score:", macro_avg_f1)

Macro-averaged F1-score: 0.6008183241650797


In [246]:
with open('predictions.txt', 'w') as out:
    for uid, text_scores in zip(range(len(test_tokens)), clean_f1(0.5)[3]):
        out.write(f'{str(uid)}\t{str(text_scores)}\n')

----------------------------------BERT MODELL 5 EPOCH------------------------------

In [12]:
def padding(tokens, max_len=maxlen):
    tokens_len = len(tokens)
    pad_len = max(0, max_len - tokens_len)
    return (
        pad_sequences([tokens], maxlen=max_len, truncating="post", padding="post", dtype="int"),
        np.concatenate([np.ones(tokens_len, dtype="int"), np.zeros(pad_len, dtype="int")], axis=0)
    )

def token_and_mask(tokens):
    token_ids, masks = [], []

    for x in tokens:
        token_id, mask = padding(tokenizer.convert_tokens_to_ids(x))
        token_ids.append(token_id[0])
        masks.append(mask)

    token_ids = np.array(token_ids)
    masks = np.array(masks)

    return token_ids, masks

train_token_ids, train_masks = token_and_mask(train_tokens)
test_token_ids, test_masks = token_and_mask(test_tokens)


train_token_labels = list(map(lambda t: [0] + t[:maxlen - 2] + [0], df_2['token_labels']))

train_y = pad_sequences(train_token_labels, maxlen=maxlen, truncating="post", padding="post")[:, :, None]
train_y.shape, np.mean(train_y)


class BertClassifier(nn.Module):

    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.hidden = nn.Linear(bert.config.hidden_size, 64)
        self.hidden_activation = nn.LeakyReLU(0.1)
        self.output = nn.Linear(64, 1)
        self.output_activation = nn.Sigmoid()

    def forward(self, input_ids, attention_mask, labels=None):

        outputs = self.bert(input_ids, attention_mask=attention_mask)
        cls_output = outputs[0]
        cls_output = self.hidden(cls_output)
        cls_output = self.hidden_activation(cls_output)
        cls_output = self.output(cls_output)
        cls_output = self.output_activation(cls_output)
        criterion = nn.BCELoss()
        loss = 0
        if labels is not None:
            loss = criterion(cls_output, labels.float())
        return loss, cls_output

device = torch.device("cpu")
model = BertClassifier(BertModel.from_pretrained('bert-base-uncased')).to(device)

BATCH_SIZE = 16

train_dataset = TensorDataset(
    torch.tensor(train_token_ids), torch.tensor(train_masks), torch.tensor(train_y))
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(torch.tensor(test_token_ids), torch.tensor(test_masks))
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

optimizer = Adam(model.parameters(), lr=3e-6)
torch.cuda.empty_cache()


EPOCHS = 5
loss = nn.BCELoss()
total_len = len(train_token_ids)
batch_losses = []
model.train()

for epoch_num in range(EPOCHS):
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(tqdm(train_dataloader)):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        loss, _ = model(input_ids=token_ids, attention_mask=masks, labels=labels)

        train_loss += loss.item()

        model.zero_grad()
        loss.backward()

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()

        batch_losses.append(train_loss / (step_num + 1))


model.eval()

test_tokens = []
test_attention_masks = []
test_preds = []

with torch.no_grad():
    for step_num, batch_data in enumerate(tqdm(test_dataloader)):
        token_ids, masks = tuple(t.to(device) for t in batch_data)
        _, output = model(input_ids=token_ids, attention_mask=masks)
        test_tokens += token_ids.tolist()
        test_attention_masks += masks.tolist()
        test_preds += output[:, :, 0].tolist()





Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 125/125 [05:42<00:00,  2.74s/it]


In [31]:
def f1_scoremetric(predictions, gold, n):
    predictions = set(predictions)
    gold = set(gold)
    if len(gold) == 0:
        return {
            'f1': 1 if len(predictions)==0 else 0
        }
    nom = 2*len(predictions.intersection(gold))
    denom = len(predictions)+len(gold)
    if len(gold) == n:
        return {
            'f1': nom/denom,
        }
    return {
        'f1': nom/denom,
    }

def clean_f1(threshold):
    n = len(test_tokens)
    y_pred = []
    all_spans = []
    f1=0
    for i in range(n):
        reconstructed_text = ''
        spans = []
        original_text = df_test2.iloc[i]['text']
        original_text = original_text.replace('É', 'e')
        original_text = original_text.replace('\u200b', '')
        original_text = original_text.replace('ü', 'u')
        original_l = len(original_text)
        idx = 0
        tokens = test_tokens[i]
        n_tokens = len(tokens)
        tokens = tokenizer.convert_ids_to_tokens(tokens)

        j = 1
        last_token = None
        prev_token_toxic = False
        prev_token_prob = 0
        prev_idx = -1
        while j < n_tokens:
            while idx < original_l and original_text[idx].isspace():
                reconstructed_text += original_text[idx]
                idx += 1
            word = tokens[j]
            if word == '[SEP]':
                last_token = j - 1
                break
            if ord(original_text[idx]) == 65039:
                print('Problematic char at', idx, i)
                reconstructed_text += original_text[idx]
                idx += 1
            if word == '[UNK]':
                reconstructed_text += original_text[idx]
                idx += 1
                j += 1
                continue
            max_toxic_prob = test_preds[i][j]
            while j < n_tokens - 1 and tokens[j + 1].startswith('##'):
                j += 1
                word += tokens[j][2:]
                max_toxic_prob = max(max_toxic_prob, test_preds[i][j])
            word_l = len(word)
            y_pred += [min(prev_token_prob, max_toxic_prob) for _ in range(prev_idx + 1, idx)]
            if min(prev_token_prob, max_toxic_prob) >= threshold:
                spans += list(range(prev_idx + 1, idx))
            y_pred += [max_toxic_prob for _ in range(word_l)]
            is_toxic = (max_toxic_prob >= threshold)
            prev_token_prob = max_toxic_prob
            if word == original_text[idx: idx + word_l].lower():
                reconstructed_text += word
                if is_toxic:
                    spans += list(range(idx, idx + word_l))
                idx += word_l
                prev_idx = idx - 1
                prev_token_toxic = is_toxic
            else:
                print(word, original_text[idx: idx + word_l])
            j += 1

        while idx < original_l and original_text[idx].isspace():
            reconstructed_text += original_text[idx]
            idx += 1
        y_pred += [prev_token_prob for _ in range(prev_idx + 1, idx)]

        if reconstructed_text != original_text.lower():
            print('ISSUE', i)
        else:
            metrics = f1_scoremetric(spans, eval(df_test2.iloc[i]['index']), len(original_text))
            f1 += metrics['f1']

        all_spans.append(spans)

    return f1 / n, all_spans

In [33]:
thresholds = [0.01 * x for x in range(100)]
f1_scores=[]
for x in tqdm(thresholds):
    scores = clean_f1(x)
    f1_scores.append(scores[0])

100%|██████████| 100/100 [01:12<00:00,  1.39it/s]


In [58]:
df_test2["predicted"]=""
df_test2["predicted"] = [text_scores for uid, text_scores in zip(range(len(test_tokens)), clean_f1(0.57)[1])] #0.57
df_test2

,index,text,toxic,bert_tokens,predicted
0,"[84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...",That's right. They are not normal. And I am st...,0,"[that, ', s, right, ., they, are, not, normal,...","[118, 119, 120, 121, 122, 123, 133, 134, 135, ..."
1,"[81, 82, 83, 84, 85, 86]","""Watch people die from taking away their healt...",0,"["", watch, people, die, from, taking, away, th...","[81, 82, 83, 84, 85, 86]"
2,[],tens years ago i contacted the PDR and suggest...,0,"[tens, years, ago, i, contacted, the, pd, ##r,...","[483, 484, 485, 486, 487, 488, 489, 490]"
3,[],The parallels between the ANC and the Sicilian...,0,"[the, parallels, between, the, an, ##c, and, t...","[413, 414, 415, 416, 417, 418, 419, 420]"
4,[],Intel Community: ‘How can we work for a Presid...,0,"[intel, community, :, ‘, how, can, we, work, f...","[663, 664, 665, 666, 667, 668, 669]"
...,...,...,...,...,...
1995,"[4, 5, 6, 7, 8, 70, 71, 72, 73, 74, 75, 76, 77...",hey loser change your name to something more a...,0,"[hey, loser, change, your, name, to, something...","[4, 5, 6, 7, 8, 70, 71, 72, 73, 74, 75, 76, 77]"
1996,"[23, 24, 25, 26, 27]",And you are a complete moron who obviously doe...,0,"[and, you, are, a, complete, mor, ##on, who, o...","[23, 24, 25, 26, 27, 80, 81, 82, 83, 84, 85, 8..."
1997,"[157, 158, 159, 160, 161, 162, 163, 164, 165, ...",Such vitriol from the left. Who would have th...,0,"[such, vi, ##tri, ##ol, from, the, left, ., wh...","[157, 158, 159, 160, 161, 162, 163, 164, 165, ..."
1998,[],It is now time for most of you to expand your ...,0,"[it, is, now, time, for, most, of, you, to, ex...",[]


In [59]:
predicted = df_test2["predicted"].tolist()
actual = df_test2["index"].tolist()

f1_scores2 = []
for pred, act in zip(predicted, actual):
    f1_scores2.append(f1(pred, act))

macro_avg_f1 = np.mean(f1_scores2)

print("Macro-averaged F1-score:", macro_avg_f1)
txt = "F1 score: {F1_score:.2f}%"
print(txt.format(F1_score = macro_avg_f1*100))

Macro-averaged F1-score: 0.6414839024042502
F1 score: 64.15%
